In [1]:
import pandas as pd
import numpy as np
import pickle

In [29]:
with open("ticker_data_augmented.pkl", "rb") as file:
    loaded_data = pickle.load(file)

stocks = list(loaded_data.keys())[:-1]

In [30]:
caps_threshold = {
    'Risk' : [1, 2, 3],
    'Threshold' : [10000000000, 2000000000, 250000000]
}

mkt_caps = {}

for stock in stocks:
    mkt_cap = loaded_data[stock].mean()['Market_Cap']
    for i, threshold in enumerate(caps_threshold['Threshold'], start=1):
        if mkt_cap >= threshold:
            mkt_caps[stock] = i
            break
    else: mkt_caps[stock] = 4

In [41]:
def screening_level_1(data, measure_name):
    measure = {}

    for stock in stocks:
        stock_measure = data[stock].mean()[measure_name]
        measure[stock] = stock_measure / np.sqrt(252)

    measure_vals = list(measure.values())
    measure_vals = [x for x in measure_vals if not np.isnan(x)]
    quartiles = {
        'Q' : [1, 2, 3, 4],
        'Measure' : [np.quantile(measure_vals, x) for x in [0.25, 0.5, 0.75, 1.0]]
    }

    quart = pd.DataFrame(quartiles).set_index('Q')

    risk_levels = {}

    for stock in stocks:
        for i, threshold in enumerate(quart['Measure'], start=1):
            if measure[stock] <= threshold:
                risk_levels[stock] = i
                break
        else: risk_levels[stock] = np.nan

    return risk_levels

In [42]:
vol_risk = screening_level_1(loaded_data, 'Annualized Vol')

In [43]:
vol_weight = 0.5
mkt_cap_weight = 0.5

weighted_risk = {
    key: round(vol_weight * vol_risk[key] + mkt_cap_weight * mkt_caps[key], 1) for key in mkt_caps
}

In [44]:
risk = {
    1: [],
    2: [],
    3: [],
    4: []
}

for stock in stocks:
    if weighted_risk[stock] == 1.0:
        risk[1].append(stock)
    elif weighted_risk[stock] == 1.5:
        risk[2].append(stock)
    elif weighted_risk[stock] == 2.0:
        risk[3].append(stock)
    elif weighted_risk[stock] == 2.5:
        risk[4].append(stock)

In [45]:
len(risk[1])

123

In [46]:
len(risk[2])

116

In [47]:
len(risk[3])

121

In [48]:
len(risk[4])

128

In [75]:
std = {}
downside = {}
dividend_yield = {}
alpha = {}
debt_equity = {}
eps_growth_vol = {}
earnings_price = {}
book_val_price = {}
roe = {}
beta = {}

for stock in stocks:
    std[stock] = loaded_data[stock].std()['Return']
    
    downside[stock] = np.sqrt((np.minimum(0, loaded_data[stock]['Return'])**2).mean())
    
    dividend_yield[stock] = (loaded_data[stock]['Last_Dividend'] / loaded_data[stock]['Adj Close']).mean() * 4
    
    alpha[stock] = (loaded_data[stock]['Return'] - loaded_data['SPY']['Return']).mean()
    
    debt_equity[stock] = loaded_data[stock]['Debt_Ratio'].mean()
    
    g = (loaded_data[stock]['EPS'] - loaded_data[stock]['EPS'].shift(1)) / loaded_data[stock]['EPS'].shift(1)
    eps_growth_vol[stock] = np.sqrt(((g - g.mean())**2).mean())
    
    earnings_price[stock] = (loaded_data[stock]['EPS'] / loaded_data[stock]['Adj Close']).mean()

    shares_outstanding = loaded_data[stock]['Market_Cap'] / loaded_data[stock]['Adj Close']
    book_val_per_share = loaded_data[stock]['Book_Value'] / shares_outstanding
    book_val_price[stock] = (book_val_per_share / loaded_data[stock]['Adj Close']).mean()
    
    net_income = shares_outstanding * loaded_data[stock]['EPS']
    roe[stock] = net_income / loaded_data[stock]['Book_Value']

    cov = np.cov(loaded_data[stock]['Return'].dropna(), loaded_data['SPY']['Return'].dropna().tail(len(loaded_data[stock]['Return'].dropna())))
    var = loaded_data[stock].var()['Return']
    beta[stock] = cov / var